Action1	

"数据集：天猫用户复购预测

使用传统机器学习完成预测
使用Attention机制的DNN模型完成预测
分享交流你们的经验

Score > 0.66"	

https://tianchi.aliyun.com/competition/entrance/231576/introduction		

"1、完成代码（10points）

2、分享经验（10points）

3、使用Attention模型（10points）

4、提交结果>0.66（10points）"

In [18]:
import gc
import pandas as pd
# 用户行为，使用format1进行加载
# 加载全量样本

user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
train_data1 = pd.read_csv('./data_format1/train_format1.csv')
submission = pd.read_csv('./data_format1/test_format1.csv')
"""

# 加载小样本
user_log = pd.read_csv('./data_format1_small/sample_user_log.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./data_format1_small/sample_user_info.csv')
train_data1 = pd.read_csv('./data_format1_small/train.csv')
"""

submission = pd.read_csv('./data_format1_small/test.csv')
train_data = pd.read_csv('./data_format2/train_format2.csv')

train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
#print(matrix)

matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
# 1 for <18; 2 for [18,24]; 3 for [25,29]; 4 for [30,34]; 5 for [35,39]; 6 for [40,49]; 7 and 8 for >= 50; 0 and NULL for unknown
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
del user_info, train_data1
gc.collect()
print(matrix)

# User特征处理
groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
# 对于每个user_id 不重复的item_id的数量 => u2
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
# 对于每个user_id 不重复的cat_id的数量 => u3
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
# 对于每个user_id 计算time_stamp的最小时间 => F_time, 最大时间max => L_time
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
#print(matrix)

# 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
#print(matrix)

# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
#print(matrix)

#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
print(matrix)

# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()


        user_id  merchant_id label origin  age_range  gender
0         34176         3906   0.0  train          6       0
1         34176          121   0.0  train          6       0
2         34176         4356   1.0  train          6       0
3         34176         2217   0.0  train          6       0
4        230784         4818   0.0  train          0       0
...         ...          ...   ...    ...        ...     ...
266915    47231         1748   nan   test          0       0
266916    59519          798   nan   test          3       0
266917   263039          639   nan   test          2       1
266918   263039         3954   nan   test          2       1
266919   423551         2954   nan   test          4       0

[266920 rows x 6 columns]


/Users/Leigh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
/Users/Leigh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:100: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


        user_id  merchant_id label origin    u1   u2  u3   u4   u5        u6  \
0         34176         3906   0.0  train   451  256  45  109  108  5.833333   
1         34176          121   0.0  train   451  256  45  109  108  5.833333   
2         34176         4356   1.0  train   451  256  45  109  108  5.833333   
3         34176         2217   0.0  train   451  256  45  109  108  5.833333   
4        230784         4818   0.0  train    54   31  17   20   19  5.166667   
...         ...          ...   ...    ...   ...  ...  ..  ...  ...       ...   
266915    47231         1748   nan   test   128   97  28   39   40  5.816667   
266916    59519          798   nan   test  1286  540  55   93   96  6.000000   
266917   263039          639   nan   test     9    8   7    7    7  5.783333   
266918   263039         3954   nan   test     9    8   7    7    7  5.783333   
266919   423551         2954   nan   test   197   85  36   39   40  5.916667   

        ...  age_2  age_3  age_4  age_5

12

In [19]:
# 使用机器学习工具
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import xgboost as xgb
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)

# 使用XGBoost
model = xgb.XGBClassifier(
    max_depth=20,
    n_estimators=2000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42    
)
model.fit(
    X_train, y_train,
    eval_metric='auc', eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10
)


model.fit(X_train, y_train)

"""
prob = model.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('prediction.csv', index=False)
"""

[0]	validation_0-auc:0.628148	validation_1-auc:0.614506
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.651356	validation_1-auc:0.635952
[2]	validation_0-auc:0.657154	validation_1-auc:0.64044
[3]	validation_0-auc:0.6612	validation_1-auc:0.643988
[4]	validation_0-auc:0.666103	validation_1-auc:0.644589
[5]	validation_0-auc:0.6709	validation_1-auc:0.64829
[6]	validation_0-auc:0.672185	validation_1-auc:0.649116
[7]	validation_0-auc:0.674066	validation_1-auc:0.651778
[8]	validation_0-auc:0.676491	validation_1-auc:0.652038
[9]	validation_0-auc:0.676885	validation_1-auc:0.651995
[10]	validation_0-auc:0.677986	validation_1-auc:0.652146
[11]	validation_0-auc:0.678512	validation_1-auc:0.651484
[12]	validation_0-auc:0.679337	validation_1-auc:0.652348
[13]	validation_0-auc:0.679579	validation_1-auc:0.652225
[14]	validation_0-auc:0.679612	validation_1-auc:0.652349
[15]

[141]	validation_0-auc:0.72578	validation_1-auc:0.677034
[142]	validation_0-auc:0.726001	validation_1-auc:0.677178
[143]	validation_0-auc:0.726252	validation_1-auc:0.677311
[144]	validation_0-auc:0.726495	validation_1-auc:0.677286
[145]	validation_0-auc:0.726675	validation_1-auc:0.677339
[146]	validation_0-auc:0.72703	validation_1-auc:0.677492
[147]	validation_0-auc:0.727253	validation_1-auc:0.6775
[148]	validation_0-auc:0.727475	validation_1-auc:0.677607
[149]	validation_0-auc:0.7277	validation_1-auc:0.67757
[150]	validation_0-auc:0.727889	validation_1-auc:0.677519
[151]	validation_0-auc:0.72822	validation_1-auc:0.67755
[152]	validation_0-auc:0.728512	validation_1-auc:0.677444
[153]	validation_0-auc:0.728635	validation_1-auc:0.677508
[154]	validation_0-auc:0.728916	validation_1-auc:0.677519
[155]	validation_0-auc:0.729131	validation_1-auc:0.677401
[156]	validation_0-auc:0.729308	validation_1-auc:0.677535
[157]	validation_0-auc:0.729605	validation_1-auc:0.677547
[158]	validation_0-auc:

"\nprob = model.predict_proba(test_data)\nsubmission['prob'] = pd.Series(prob[:,1])\nsubmission.drop(['origin'], axis=1, inplace=True)\nsubmission.to_csv('prediction.csv', index=False)\n"